In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize
# Machine learning
import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv
from sklearn.preprocessing import StandardScaler
from keras.layers.advanced_activations import ReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D




# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')


In [ ]:
train_data = pd.read_csv('../input/facial-keypoints-detection/training.zip', compression='zip', header=0, sep=',', quotechar='"')
test_data = pd.read_csv('../input/facial-keypoints-detection/test.zip', compression='zip', header=0, sep=',', quotechar='"')
IdLookupTable = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv',header=0, sep=',', quotechar='"')
SampleSubmission = pd.read_csv('../input/facial-keypoints-detection/SampleSubmission.csv',header=0, sep=',', quotechar='"')





In [ ]:
train_data.head()

In [ ]:
train_data.head().T.tail()


In [ ]:
len(train_data['Image'][1])


In [ ]:
train_data.info()

In [ ]:
test_data.head()

In [ ]:
len(test_data['Image'][1])


In [ ]:
test_data.info()

In [ ]:
IdLookupTable.head()

In [ ]:
IdLookupTable.info()

In [ ]:
SampleSubmission.head()

# Check Missing Data


### Train

In [ ]:

display(train_data.isnull().sum())


In [ ]:
train_data.isnull().sum().plot(kind='bar',color = 'pink')
plt.title("Missing Data ")
plt.show()

In [ ]:
train_data.fillna(method = 'ffill',inplace = True)


In [ ]:
train_data.isnull().sum().plot(kind='bar',color = 'pink')
plt.title("Missing Data ")
plt.show()

### test

In [ ]:
display(test_data.isnull().sum())


In [ ]:
test_data.isnull().sum().plot(kind='bar',color = 'pink')
plt.title("Missing Data ")
plt.show()

### IdLookupTable

In [ ]:
display(IdLookupTable.isnull().sum())


# Visualize Data

In [ ]:
Vis = []

for i in range(len(train_data)):
  Vis.append(train_data['Image'][i].split(' '))


#### prepare data x train

In [ ]:
array_float = np.array(Vis, dtype='float')

X_train = array_float.reshape(-1,96,96,1)

#### show Photo

In [ ]:
photo_visualize = array_float[1].reshape(96,96)

plt.imshow(photo_visualize, cmap='gray')
plt.title("Visualize Image")
plt.show()

#### Facial Keypoints

In [ ]:

Facial_Keypoints_Data = train_data.drop(['Image'], axis=1)
facial_pnts = []

for i in range(len(Facial_Keypoints_Data)):
  facial_pnts.append(Facial_Keypoints_Data.iloc[i])
  

##### prepare data y train

In [ ]:
training_data = train_data.drop('Image',axis = 1)

y_train = []
for i in range(len(train_data)):
    y = training_data.iloc[i,:]
    y_train.append(y)
    
    
y_train = np.array(y_train,dtype = 'float')

In [ ]:

facial_pnts_float = np.array(facial_pnts, dtype='float')


#### Show Photo image with facial points

In [ ]:
photo_visualize_pnts = Facial_Keypoints_Data.iloc[1].values

plt.imshow(photo_visualize, cmap='gray')
plt.scatter(photo_visualize_pnts[0::2], photo_visualize_pnts[1::2], c='Pink', marker='*')
plt.title("Image with Facial Keypoints")
plt.show()

# Prepare and Split Data

In [ ]:
train_data.shape

# Build Model

## Keras CNN 

In [ ]:
model = Sequential()


# layer 1
model.add(Convolution2D(32, (3,3), activation = 'relu', padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

# layer 2

model.add(Convolution2D(32, (3,3), activation = 'relu', padding='same', use_bias=False))
model.add(MaxPool2D(pool_size=(2, 2)))

# layer 3

model.add(Convolution2D(64, (3,3), activation = 'relu', padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

# layer 4

model.add(Convolution2D(128, (3,3), activation = 'relu', padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['acc'])

# train Data

In [ ]:
model.fit(X_train,y_train,epochs = 5,batch_size = 32,validation_split = 0.2)

## prepare data test

In [ ]:
test_images = []
for i in range(len(test_data)):
    item = test_data['Image'][i].split(' ')
    test_images.append(item)

In [ ]:
array_float_test = np.array(test_images,dtype = 'float')
X_test = array_float_test.reshape(-1,96,96,1) 

# predict

In [ ]:
predict = model.predict(X_test)


# submission

In [ ]:
# from IdLookupTable table 
FeatureName = list(IdLookupTable['FeatureName'])
ImageId = list(IdLookupTable['ImageId']-1)
RowId = list(IdLookupTable['RowId'])


# predict results
predict = list(predict)


Data = []
for i in list(FeatureName):
    Data.append(FeatureName.index(i))
    
    
Data_Pre = []
for x,y in zip(ImageId,Data):
    Data_Pre.append(predict[x][y])
    
    





RowId = pd.Series(RowId,name = 'RowId')
Location = pd.Series(Data_Pre,name = 'Location')

submission = pd.concat([RowId,Location],axis = 1)
submission.to_csv('./Submit.csv',index=False)

